In [0]:
#import georasters as gs
import numpy as np
import gdal
from osgeo import gdal_array
import os
import sys
from glob import glob
#import geopandas as gpd
import math 
#import rasterio as rio
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
from scipy.constants import pi,h,c,Boltzmann

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = "/content/drive/My Drive/Colab Notebooks/SOP/Landsat 8 - 2018"

## FUNCTIONS FOR READING GEOSPATIAL IMAGES





In [0]:
from __future__ import division
import numpy as np
from osgeo import gdal, gdalnumeric, ogr, osr, gdal_array
from gdalconst import GA_ReadOnly
from skimage.measure import block_reduce
from skimage.transform import resize
import skimage.graph as graph
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Function to read the original file's projection:
def get_geo_info(filename, band=1):
    ''' Gets information from a Raster data set
    '''
    sourceds = gdal.Open(filename, GA_ReadOnly)
    ndv = sourceds.GetRasterBand(band).GetNoDataValue()
    xsize = sourceds.RasterXSize
    ysize = sourceds.RasterYSize
    geot = sourceds.GetGeoTransform()
    projection = osr.SpatialReference()
    projection.ImportFromWkt(sourceds.GetProjectionRef())
    datatype = sourceds.GetRasterBand(band).DataType
    datatype = gdal.GetDataTypeName(datatype)
    return ndv, xsize, ysize, geot, projection, datatype

In [0]:
def load_tiff(file):
    """
    Load a geotiff raster keeping ndv values using a masked array

    Usage:
            data = load_tiff(file)
    """
    ndv, xsize, ysize, geot, projection, datatype = get_geo_info(file)
    data = gdalnumeric.LoadFile(file)
    data = np.ma.masked_array(data, mask=data == ndv, fill_value=ndv)
    return data


## TEMPERATURE AND RAINFALL DETAILS

In [0]:
data_final=[]

Rainfall_actual={"Meerut":1,"Ludhiana":5.6,"MATHURA":40.9,"Alwar":29.1,"JAIPUR":28.2,"NAGAUR":10.7,"Hanumangarh":0.4,"Baran":287.9,"Bhilwara":313.9,"Chittorgarh":202.6,"Kannauj":354.1,"Satara":264.7,"Hubei China":39.6,"AMRITSAR":14.6,"SIKAR":0.4}

Temperature_actual={"Meerut":29.36,"Ludhiana":29.384,"MATHURA":33.89,"Alwar":33.43,"JAIPUR":33.56,"NAGAUR":34.61,"Hanumangarh":31.46,"Baran":35.9,"Bhilwara":35.18,"Chittorgarh":34.58,"Kannauj":38.15,"Satara":25.81,"Hubei China":24.76,"AMRITSAR":25.3,"SIKAR":26.04}

crop={"Meerut":"Wheat","Ludhiana":"Wheat","MATHURA":"Bajra","Alwar":"Mustard","JAIPUR":"Barley","NAGAUR":"Bajra","Hanumangarh":"Wheat","Baran":"Soyabean","Bhilwara":"Maize","Chittorgarh":"Maize","Kannauj":"Maize","Satara":"Soyabean","Hubei China":"Mustard","AMRITSAR":"Barley","SIKAR":"Barley"}

## SATELLITE BASED CONSTANTS

In [0]:
#Sun elevation
SUN_ELEVATION ={0:52.42314679,1:37.65849090,2:49.79561784,3:53.47774513,4:49.26508403,5:68.83716345,6:68.98267073,7:68.60600422,8:68.83617595,9:67.08188409,10:53.47774513,11:44.49391638,12:45.79375446,13:51.39649683}
#sine_se = round(math.sin(math.radians(SUN_ELEVATION)),1)
#Radiance 
RADIANCE_MAXIMUM_BAND_1 = 783.81287
RADIANCE_MINIMUM_BAND_1 = -64.72752
RADIANCE_MAXIMUM_BAND_2 = 802.63391
RADIANCE_MINIMUM_BAND_2 = -66.28177
RADIANCE_MAXIMUM_BAND_3 = 739.62030
RADIANCE_MINIMUM_BAND_3 = -61.07809
RADIANCE_MAXIMUM_BAND_4 = 623.68951
RADIANCE_MINIMUM_BAND_4 = -51.50448
RADIANCE_MAXIMUM_BAND_5 = 381.66687
RADIANCE_MINIMUM_BAND_5 = -31.51818
RADIANCE_MAXIMUM_BAND_6 = 94.91708
RADIANCE_MINIMUM_BAND_6 = -7.83828
RADIANCE_MAXIMUM_BAND_7 = 31.99214
RADIANCE_MINIMUM_BAND_7 = -2.64192
RADIANCE_MAXIMUM_BAND_8 = 705.84491
RADIANCE_MINIMUM_BAND_8 = -58.28890
RADIANCE_MAXIMUM_BAND_9 = 149.16406
RADIANCE_MINIMUM_BAND_9 = -12.31802
RADIANCE_MAXIMUM_BAND_10 = 22.00180
RADIANCE_MINIMUM_BAND_10 = 0.10033
RADIANCE_MAXIMUM_BAND_11 = 22.00180
RADIANCE_MINIMUM_BAND_11 = 0.10033

In [0]:
#Reflectance
REFLECTANCE_MAXIMUM_BAND_1 = 1.210700
REFLECTANCE_MINIMUM_BAND_1 = -0.099980
REFLECTANCE_MAXIMUM_BAND_2 = 1.210700
REFLECTANCE_MINIMUM_BAND_2 = -0.099980
REFLECTANCE_MAXIMUM_BAND_3 = 1.210700
REFLECTANCE_MINIMUM_BAND_3 = -0.099980
REFLECTANCE_MAXIMUM_BAND_4 = 1.210700
REFLECTANCE_MINIMUM_BAND_4 = -0.099980
REFLECTANCE_MAXIMUM_BAND_5 = 1.210700
REFLECTANCE_MINIMUM_BAND_5 = -0.099980
REFLECTANCE_MAXIMUM_BAND_6 = 1.210700
REFLECTANCE_MINIMUM_BAND_6 = -0.099980
REFLECTANCE_MAXIMUM_BAND_7 = 1.210700
REFLECTANCE_MINIMUM_BAND_7 = -0.099980
REFLECTANCE_MAXIMUM_BAND_8 = 1.210700
REFLECTANCE_MINIMUM_BAND_8 = -0.099980
REFLECTANCE_MAXIMUM_BAND_9 = 1.210700
REFLECTANCE_MINIMUM_BAND_9 = -0.099980
REFLECTANCE_MULT_BAND=2.0000E-05
REFLECTANCE_MULT_BAND_1 = 2.0000E-05
REFLECTANCE_MULT_BAND_2 = 2.0000E-05
REFLECTANCE_MULT_BAND_3 = 2.0000E-05
REFLECTANCE_MULT_BAND_4 = 2.0000E-05
REFLECTANCE_MULT_BAND_5 = 2.0000E-05
REFLECTANCE_MULT_BAND_6 = 2.0000E-05
REFLECTANCE_MULT_BAND_7 = 2.0000E-05
REFLECTANCE_MULT_BAND_8 = 2.0000E-05

REFLECTANCE_ADD_BAND = -0.100000

In [0]:
#Calibrations
QUANTIZE_CAL_MAX_BAND_1 = 65535
QUANTIZE_CAL_MIN_BAND_1 = 1
QUANTIZE_CAL_MAX_BAND_2 = 65535
QUANTIZE_CAL_MIN_BAND_2 = 1
QUANTIZE_CAL_MAX_BAND_3 = 65535
QUANTIZE_CAL_MIN_BAND_3 = 1
QUANTIZE_CAL_MAX_BAND_4 = 65535
QUANTIZE_CAL_MIN_BAND_4 = 1
QUANTIZE_CAL_MAX_BAND_5 = 65535
QUANTIZE_CAL_MIN_BAND_5 = 1
QUANTIZE_CAL_MAX_BAND_6 = 65535
QUANTIZE_CAL_MIN_BAND_6 = 1
QUANTIZE_CAL_MAX_BAND_7 = 65535
QUANTIZE_CAL_MIN_BAND_7 = 1
QUANTIZE_CAL_MAX_BAND_8 = 65535
QUANTIZE_CAL_MIN_BAND_8 = 1
QUANTIZE_CAL_MAX_BAND_9 = 65535
QUANTIZE_CAL_MIN_BAND_9 = 1
QUANTIZE_CAL_MAX_BAND_10 = 65535
QUANTIZE_CAL_MIN_BAND_10 = 1
QUANTIZE_CAL_MAX_BAND_11 = 65535
QUANTIZE_CAL_MIN_BAND_11 = 1

In [0]:
#Thermal Constants

K1_CONSTANT_BAND_10 = 774.8853
K2_CONSTANT_BAND_10 = 1321.0789
K1_CONSTANT_BAND_11 = 480.8883
K2_CONSTANT_BAND_11 = 1201.1442

In [0]:
#Constants for thermal bands BAND 10
Lmin=RADIANCE_MINIMUM_BAND_10
Lmax=RADIANCE_MAXIMUM_BAND_10
QCALmax=QUANTIZE_CAL_MAX_BAND_10
QCALmin=QUANTIZE_CAL_MIN_BAND_10
e=0.95
s=Boltzmann
K1=K1_CONSTANT_BAND_10
K2=K2_CONSTANT_BAND_10
c1 = 1.4388*pow(10,-2)
c2=h*c/s
wavelength=11*pow(10,-6)

## CALCULATING IMAGE BASED FACTORS

In [0]:
def calculate_spectral_radiance(data):
    L=np.where(data!=np.nan,Lmin+(((Lmax-Lmin)/(QCALmax-QCALmin))*(data-QCALmin)),np.nan)
    return np.nan_to_num(L)

In [0]:
def calculate_surface_reflectance(data,i):
    sine_se = round(math.sin(math.radians(SUN_ELEVATION[i])),1)
    SR = np.where(
      (data) == 0,
      0.,
      (REFLECTANCE_MULT_BAND_2*data + REFLECTANCE_ADD_BAND)/sine_se
      )
    return SR

In [0]:
def calculate_surface_reflectance_boa(data,i):
    sine_se = round(math.sin(math.radians(SUN_ELEVATION[i])),1)
    SR = np.where(data==0.,0.,(REFLECTANCE_MULT_BAND*data+REFLECTANCE_ADD_BAND)/sine_se)
    a = SR[SR!=np.amin(SR)]
    if(a.shape[0]==0):
        min_SR=np.amin(SR)
    else:
        min_SR=np.amin(SR[SR!=np.amin(SR)])
    SR=SR-min_SR
    return SR

In [0]:
def calculate_clay_content(b3,b5,b7,i):
    b3_sr=calculate_surface_reflectance_boa(b3,i)
    b5_sr=calculate_surface_reflectance_boa(b5,i)
    b7_sr=calculate_surface_reflectance_boa(b7,i)
    clay_content=281 - 3*b7_sr + 9.4*b3_sr + 1.1*b5_sr
    return clay_content
    

In [0]:
def calculate_sand_content(b3,b5,b7,i):
    b3_sr=calculate_surface_reflectance_boa(b3,i)
    b5_sr=calculate_surface_reflectance_boa(b5,i)
    b7_sr=calculate_surface_reflectance_boa(b7,i)
    sand_content=585.5 + 29.5*b7_sr - 12.8*b5_sr - 5.3*b3_sr
    return sand_content
    

In [0]:
def calculate_soil_pH(b2,b3,b4,b6,b7,i):
    b2_sr=calculate_surface_reflectance_boa(b2,i)
    b3_sr=calculate_surface_reflectance_boa(b3,i)
    b4_sr=calculate_surface_reflectance_boa(b4,i)
    b6_sr=calculate_surface_reflectance_boa(b6,i)
    b7_sr=calculate_surface_reflectance_boa(b7,i)
    
    pH = np.where(
        (b2_sr + b3_sr + b4_sr + b6_sr + b7_sr) == 0.,
        np.nan,
        6.232-59.439*b2_sr - 89.326*b3_sr + 136.721*b4_sr+5.612*b6_sr-25.603*b7_sr
    )
    return pH
    

In [0]:
def calculate_NDVI(Red,NIR,i):
    NIR = calculate_surface_reflectance_boa(NIR,i)
    Red = calculate_surface_reflectance_boa(Red,i)
    NDVI = np.where(
            (NIR + Red) == 0.,
            np.nan,
            (NIR - Red)/(NIR + Red)
    )
    return NDVI

In [0]:
def calculate_NDWI(Green, NIR,i):
    NIR=calculate_surface_reflectance_boa(NIR,i)
    Green=calculate_surface_reflectance_boa(Green,i)
    NDWI = np.where(
            (NIR + Green) == 0.,
            np.nan,
            (NIR - Green)/(NIR + Green)
    )
    return NDWI

In [0]:
def calculate_LST(data):
    L=calculate_spectral_radiance(data)
    #print(L)
    Tb = (K2 / (np.log(K1 * e / L + 1)))
    LST = Tb / [1 + (wavelength * Tb / c2) * np.log(e)]
    return LST

In [0]:
def calculate_constants(NIR,Red,Thermal,i):
    NDVI=calculate_NDVI(Red,NIR,i).flatten()
    LST=calculate_LST(Thermal)[0]
    #print(NDVI)
    # print(LST.shape)
    ndvi_max_ind=np.argwhere(NDVI==np.amax(NDVI))
    ndvi_min_ind=np.argwhere(NDVI==np.amin(NDVI))
    # print(ndvi_max_ind)
    # print(ndvi_min_ind)
    if(ndvi_max_ind.size==0):
        return 0,0,0,0
    LST_up_2=np.nanmax(LST.flatten()[ndvi_max_ind])
    LST_up_1=np.nanmax(LST.flatten()[ndvi_min_ind])
    LST_lo_2=np.nanmin(LST.flatten()[ndvi_max_ind])
    LST_lo_1=np.nanmin(LST.flatten()[ndvi_min_ind])
    ndvi_max=np.nanmax(NDVI)
    ndvi_min=np.nanmin(NDVI)
    a1=(LST_up_2-LST_up_1)/(ndvi_max-ndvi_min)
    b1=LST_up_1-(a1)*ndvi_min
    a2=(LST_lo_2-LST_lo_1)/(ndvi_max-ndvi_min)
    b2=LST_lo_1-(a2)*ndvi_min
    # print(a1)
    # print(b1)
    # print(a2)
    # print(b2)

    return a1,b1,a2,b2

In [0]:
def calculate_SMI(Red,NIR,Thermal,a1,b1,a2,b2,i):
    NDVI=calculate_NDVI(Red,NIR,i)
    LST=calculate_LST(Thermal)[0]
    #a1,b1,a2,b2=scatter_plot(data)
    LSTmax = a1 * NDVI + b1 
    LSTmin = a2 * NDVI + b2
    SMI = (LSTmax - LST) / (LSTmax - LSTmin)
    print(SMI)
    return SMI

In [0]:
def soil_salinity(b5,b6,i):
    SSSI = np.where(
        (b5 + b6) == 0.,
        np.nan,
        (QUANTIZE_CAL_MAX_BAND_5 - b5 - b6)/(QUANTIZE_CAL_MAX_BAND_5 + b5 + b6)
    )     
    #return 122.87*pow(SSSI,2) - 54.409*SSSI + 5.6651
    return SSSI

In [0]:
def getPhosphorusContent(b2,b3,b4,i):
    TM2=np.nanmean(calculate_surface_reflectance_boa(b2,i))
    TM3=np.nanmean(calculate_surface_reflectance_boa(b3,i))
    TM4=np.nanmean(calculate_surface_reflectance_boa(b4,i))
    temp=-21.45*(TM4/TM3)-14.42*(TM2/TM4)+42.99*(TM2)+27.1

    Pcontent=np.exp(temp)
    return Pcontent
    

In [0]:
# add mineral content procedures

In [0]:
def findlandtype(ndwi):
    if ndwi<=-0.11:
        return "Dry"
    elif ndwi<=-0.08:
        return "Semi dry"
    elif ndwi<=0.08:
        return "Semi wet"
    else:
        return "Wet land"
    
def getrainfallMM(areaname):
        return Rainfall_actual[areaname]
    
def gettempCel(areaname):
    return Temperature_actual[areaname]

### MASK A TILE TO REMOVE UNNECESSARY DATA POINTS


In [0]:
def maskTile(data,i):
  NDVI=calculate_NDVI_mask(data[3],data[4],i)
  NDVI_mask = np.where((NDVI > 0.3),1.,0.)
  data1=[]
  for band in data:
    data1.append(np.multiply(band,NDVI_mask))
  return data1

In [0]:
def calculate_NDVI_mask(Red,NIR,i):
    NIR=calculate_surface_reflectance(NIR,i)
    Red=calculate_surface_reflectance(Red,i)
    NDVI = np.where(
            (NIR + Red) == 0.,
            0.,
            (NIR - Red)/(NIR + Red)
    )
    return NDVI

## GET DIAGONAL COORDINATES FOR A GIVEN TILE

In [0]:
def get_corner_cor(geodata):
    ndv, xsize, ysize, geot, projection, datatype=geodata
    ulx, xres, xskew, uly, yskew, yres=geot
    lrx = ulx + (xsize * xres)
    lry = uly + (ysize * yres)
    upper_left=(ulx,uly)
    lower_right=(lrx,lry)
    return upper_left,lower_right #upper left corner and lower right corner lat& long

## MAIN FUNCTION TO CALCULATE FEATURES FOR A GIVEN IMAGE

In [0]:
def process_landsat(bands,area_name,i):
    tile_name=os.path.basename(bands[0]).replace('.TIF','')
    #area_name='Hanumangarh'
    data=[]
    geo_data=[]
    '''
    B1=glob(os.path.join(tile,"*B1.TIF"))[0]
    B2=glob(os.path.join(tile,"*B2.TIF"))[0]
    B3=glob(os.path.join(tile,"*B3.TIF"))[0]
    B4=glob(os.path.join(tile,"*B4.TIF"))[0]
    B5=glob(os.path.join(tile,"*B5.TIF"))[0]
    B6=glob(os.path.join(tile,"*B6.TIF"))[0]
    B7=glob(os.path.join(tile,"*B7.TIF"))[0]
    B8=glob(os.path.join(tile,"*B8.TIF"))[0]
    B9=glob(os.path.join(tile,"*B9.TIF"))[0]
    B10=glob(os.path.join(tile,"*B10.TIF"))[0]
    B11=glob(os.path.join(tile,"*B11.TIF"))[0]
    '''
    
    for i in range(11):
        #bands.append(glob(os.path.join(tile,"*B{}.TIF".format(i+1)))[0])
        geo_data.append(get_geo_info(bands[i]))
        data.append(np.asarray(load_tiff(bands[i])))
        
    #print("Reflectance")
    #print(calculate_surface_reflectance_boa(data[0]))
    #print(bands)

    #Mask the tiles
    data=maskTile(data,i)

    a1,b1,a2,b2=calculate_constants(data[4],data[3],data[9],i)
    Soil_M=calculate_SMI(data[3],data[4],data[9],a1,b1,a2,b2,i)
    
    #Coordinates
    upper_left,lower_right=get_corner_cor(geo_data[0])
    
    #NDVI AND NDWI
    NDVI=np.nanmean(calculate_NDVI(data[3],data[4],i))
    NDWI=np.nanmean(calculate_NDWI(data[2],data[4],i))
    soil_moisture=np.nanmean(Soil_M)
    
    #SOIL PH, CLAY CONTENT AND SAND CONTENT
    soil_pH=np.nanmean(calculate_soil_pH(data[1],data[2],data[3],data[5],data[6],i))
    # clay_content=np.nanmean(calculate_clay_content(data[2],data[4],data[6]))
    # sand_content=np.nanmean(calculate_sand_content(data[2],data[4],data[6]))
    
    #PHOSPHORUS CONTENT
    pcontent=getPhosphorusContent(data[2],data[3],data[4],i)

    #SOIL SALINITY
    salinity= np.nanmean(soil_salinity(data[4],data[5],i))
    
    #LAND TYPE
    land_type=findlandtype(NDWI)
    
    #Rainfall
    rainfall=getrainfallMM(area_name)
    
    #Temperature
    temperature=gettempCel(area_name)
    
    #Crop
    crop_name=crop[area_name]
    
    data_excel=[tile_name,upper_left,lower_right,area_name,NDVI,NDWI,land_type,rainfall,temperature,soil_moisture,soil_pH,pcontent,salinity,crop_name]
    data_final.append(data_excel)

## GET THE CORRESPONDING 11 BANDS FOR A GIVEN TILE

In [0]:
def getBands(tilePath):
    bands=[]
    bands.append(tilePath)
    tilename=os.path.basename(tilePath)
    #print(tilePath)
    tiledir=os.path.split(tilePath)[0]
    for i in range(1,11):
        banddir=tiledir[:-1]+str(i+1)
        bandname=tilename.replace('B1','B'+str(i+1))
        #print(banddir)
        bands.append(os.path.join(banddir,bandname))
    return bands
    

## MAIN FUNCTION

In [124]:
LandsatPath=os.path.abspath(path)
districts = glob(os.path.join(LandsatPath,"*/*"))
#print(districts)
i=0
for district in districts:
    if(not os.path.isdir(district)): 
            continue
    tiles=glob(os.path.join(district,"*_TILED_1200/"))
    area_name=os.path.basename(district)
    print(area_name)
    #print(tiles)
    for tile in tiles:
        band1=glob(os.path.join(tile,"*B1"))[0]
        print(band1)
        subtiles=glob(os.path.join(band1,"*.TIF"))
        #print(subtiles)
        for image in subtiles:
            allBands=getBands(image)
            process_landsat(allBands,area_name,i)
    i=i+1


Meerut
/content/drive/My Drive/Colab Notebooks/SOP/Landsat 8 - 2018/WHEAT/Meerut/Meerut_TILED_1200/LC08_L1TP_146040_20181002_20181010_01_T1_B1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[[ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 ...
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]]
[[ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 ...
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]]
[[ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf  inf  inf]
 ...
 [ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]]
[[ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 ...
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]]
[[ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  inf ... -inf -inf -inf]
 [ inf  inf  i

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice


KeyboardInterrupt: ignored

// Trial 

In [0]:
#LandsatPath=os.path.abspath(path)
#districts=glob(os.path.join(LandsatPath,"*/*"))
#print(districts)
band1='/content/drive/My Drive/Colab Notebooks/SOP/Landsat 8 - 2018/WHEAT/Meerut/Meerut_TILED_1200/LC08_L1TP_146040_20181002_20181010_01_T1_B1'
# for district in districts:
#     if(not os.path.isdir(district)): 
#             continue
#     tiles=glob(os.path.join(district,"*_TILED/"))
#     area_name=os.path.basename(district)
  # print(area_name)
# for tile in tiles:
#     band1=glob(os.path.join(tile,"*B1"))[0]
    #print(band1)
subtiles=glob(os.path.join(band1,"*.TIF"))
print(subtiles)
for image in subtiles:
    allBands=getBands(image)
    # print(len(allBands))
    #a=load_tiff(bands[0])
    process_landsat(allBands,'Meerut')
    print("DONE")

In [0]:
subtiles

[]

## SAVE DATA TO A CSV FILE 

In [76]:
names = ['TileName','LAT/LONG upper left','LAT/LONG lower right','AreaName', 'NDVI','NDWI','LAND_TYPE(in terms of dryness)','Rainfall(in mm)','Temperature(C)','Soil Moisture','Soil PH','Phosphorous Content','Soil Salinity','Crop']
data_f=pd.DataFrame(data_final)
data_f.columns=names
print("yes")
data_f.to_csv('NDVI_indicies_landsat.csv')
print("yes")

yes
yes


## TILING USING GDAL TRANSLATE

In [0]:
tilesize_x=1200
tilesize_y=1200
#print(xsize)
#print(ysize)


In [0]:
def tile(image_path,out_path):
    ds=gdal.Open(image_path)
    band=ds.GetRasterBand(1)
    xsize=band.XSize
    ysize=band.YSize
    for i in range(0,xsize,tilesize_x):
        for j in range(0,ysize,tilesize_y):
            #print(c+1)
            com_string="gdal_translate -of GTiff -srcwin "+str(i)+" "+str(j)+" "+str(tilesize_x)+" "+str(tilesize_y)+" "+"'"+str(image_path)+"'"+" "+"'"+str(out_path)+"_"+str(i)+"_"+str(j)+".TIF'"
            #print(com_string)
            os.system(com_string)
            #c=c+1

In [0]:
LandsatPath=os.path.abspath(path)
districts=glob(os.path.join(LandsatPath,"*/*"))
#print(districts)
for district in districts:
    areaname=os.path.basename(district)
    #d_images=glob(os.path.join(district,"L*"))
    #print(d_images)
    #for direc in d_images:
    #if(not os.path.isdir(direc)): 
     #       continue
    out_=os.path.join(district,areaname+"_TILED_1200")
    if(os.path.exists(out_)):
       continue 
    else:
        os.mkdir(out_)
    bands=glob(os.path.join(district,"*.TIF")) 
    print(bands)
    for band in bands:
        band_=os.path.basename(band).replace('.TIF','')
        band_out_dir=os.path.join(out_,band_)
        #print(band_)
        #print(band_out_dir)
        os.mkdir(band_out_dir)
        outpath=band_out_dir+"/"+band_  
        #print(outpath)
        tile(band,outpath)